In [28]:
import pandas as pd
import numpy as np
import mcdm

In [29]:
shopper_df = pd.read_csv('/Users/nitanshjain/Documents/Projects/Shopper_Intent_Prediction/shopper-intent-prediction/topsis_data/topsis_long.csv')
print(shopper_df.shape)
print(shopper_df.columns)
shopper_df.head()

(360, 15)
Index(['samples', 'models', 'percentiles', 'mi_accuracy', 'mi_f1_score',
       'mi_auc', 'mrmr_accuracy', 'mrmr_f1_score', 'mrmr_auc',
       'mi_mrmr_accuracy', 'mi_mrmr_f1_score', 'mi_mrmr_auc', 'pca_accuracy',
       'pca_f1_score', 'pca_auc'],
      dtype='object')


,samples,models,percentiles,mi_accuracy,mi_f1_score,mi_auc,mrmr_accuracy,mrmr_f1_score,mrmr_auc,mi_mrmr_accuracy,mi_mrmr_f1_score,mi_mrmr_auc,pca_accuracy,pca_f1_score,pca_auc
0,subsample_7_v2.csv,lr,10,0.851360,0.847929,0.851360,0.851360,0.851163,0.851360,0.836762,0.833107,0.836762,0.842070,0.843111,0.842070
1,subsample_9_v2.csv,lr,10,0.852687,0.848049,0.852687,0.863305,0.861838,0.863305,0.842070,0.837653,0.842070,0.852356,0.851518,0.852356
2,subsample_5_v2.csv,lr,10,0.842070,0.839189,0.842070,0.843729,0.844195,0.843729,0.823822,0.821752,0.823822,0.835435,0.837163,0.835435
3,subsample_1_v2.csv,lr,10,0.847711,0.844248,0.847711,0.857664,0.856281,0.857664,0.838752,0.835700,0.838752,0.847047,0.847098,0.847047
4,subsample_10_v2.csv,lr,10,0.845720,0.841891,0.845720,0.846384,0.846128,0.846384,0.825813,0.822335,0.825813,0.836762,0.837624,0.836762


In [30]:
print(shopper_df.models.value_counts())
print(shopper_df.percentiles.value_counts())

models
lr                60
rfc               60
xgbc              60
lgbm              60
svm               60
ft_transformer    60
Name: count, dtype: int64
percentiles
10    60
20    60
30    60
50    60
75    60
90    60
Name: count, dtype: int64


In [31]:
# getting all columns that have a float64 type
shopper_df_float = shopper_df.select_dtypes(include=['float64']).columns
# shopper_df_float = ['mi_f1_score', 'mrmr_f1_score', 'mi_mrmr_f1_score', 'pca_f1_score', 'mi_auc', 'mrmr_auc', 'mi_mrmr_auc', 'pca_auc']
# shopper_df_float = ['mi_auc', 'mrmr_auc', 'mi_mrmr_auc', 'pca_auc']
# shopper_df_float = ['mi_f1_score', 'mrmr_f1_score', 'mi_mrmr_f1_score', 'pca_f1_score']
# shopper_df_float = ['mi_accuracy', 'mrmr_accuracy', 'mi_mrmr_accuracy', 'pca_accuracy']

In [32]:
def mean_models(df=shopper_df): 
    percentiles = [10, 20, 30, 50, 75, 90]
    models = ['ft_transformer', 'lr', 'rfc', 'xgbc', 'lgbm', 'svm']
    all_df = list()
    for j in percentiles:
        means = list()
        
        for i in models:
            model = df[(df.percentiles == j) & (df.models == i)]
            mean_val = list(model[shopper_df_float].mean())
            means.append(mean_val)
        means_df = pd.DataFrame(means, columns=shopper_df_float).reset_index()
        means_df['models_name'] = pd.Series(['ft_transformer', 'lr', 'rfc', 'xgbc', 'lgbm', 'svm'])
        means_df['percentiles_name'] = pd.Series([j] * 6)
        all_df.append(means_df)
        
    ranks_model_per_percentile = list()
    for i in all_df:
        i.drop(['index', 'models_name', 'percentiles_name'], axis=1, inplace=True)
        ranks_model_per_percentile.append(mcdm.rank(i.to_numpy(), s_method='TOPSIS', alt_names=models, is_benefit_x=[True]*len(shopper_df_float)))
        
    best_models = list()
    for val in ranks_model_per_percentile:
        best_models_per_percentile = list()
        # print(val)
        for i in val:
                best_models_per_percentile.append(i[0])
                
        best_models.append(best_models_per_percentile)
        
    models_df = pd.DataFrame(best_models, columns=['model1', 'model2', 'model3', 'model4', 'model5', 'model6'])
    models_df['percentiles'] = pd.Series([10, 20, 30, 50, 75, 90])
    return models_df

best_models = mean_models()
best_models

,model1,model2,model3,model4,model5,model6,percentiles
0,ft_transformer,lgbm,xgbc,rfc,svm,lr,10
1,ft_transformer,lgbm,xgbc,svm,rfc,lr,20
2,ft_transformer,lgbm,svm,xgbc,lr,rfc,30
3,ft_transformer,svm,lgbm,xgbc,lr,rfc,50
4,ft_transformer,svm,xgbc,lgbm,lr,rfc,75
5,svm,ft_transformer,xgbc,lgbm,lr,rfc,90


In [33]:
def mean_percentiles(df=shopper_df): 
    percentiles_list = [10, 20, 30, 50, 75, 90]
    models_list = ['ft_transformer', 'lr', 'rfc', 'xgbc', 'lgbm', 'svm']
    all_df = list()
    for i in models_list:
        means = list()
        for j in percentiles_list:
            model = df[(df.percentiles == j) & (df.models == i)]
            mean_val = list(model[shopper_df_float].mean())
            means.append(mean_val)
        means_df = pd.DataFrame(means, columns=shopper_df_float).reset_index()
        means_df['percentiles_name'] = pd.Series([10, 20, 30, 50, 75, 90])
        means_df['models_name'] = pd.Series([model] * 6)
        all_df.append(means_df)
        
    ranks_percentile_per_model = list()
    for i in all_df:
        i.drop(['index', 'models_name', 'percentiles_name'], axis=1, inplace=True)
        ranks_percentile_per_model.append(mcdm.rank(i.to_numpy(), s_method='TOPSIS', alt_names=percentiles_list, is_benefit_x=[True]*len(shopper_df_float)))
        
    best_percentiles = list()
    for val in ranks_percentile_per_model:
        best_percentile_per_model = list()
        # print(val)
        for i in val:
            best_percentile_per_model.append(i[0])
        best_percentiles.append(best_percentile_per_model)
        
    models_df = pd.DataFrame(best_percentiles, columns=['percentile1', 'percentile2', 'percentile3', 'percentile4', 'percentile5', 'percentile6'])
    models_df['models'] = pd.Series(['ft_transformer', 'lr', 'rfc', 'xgbc', 'lgbm', 'svm'])
    return models_df

best_percentiles = mean_percentiles()
best_percentiles
# mean_percentiles()

,percentile1,percentile2,percentile3,percentile4,percentile5,percentile6,models
0,75,30,90,50,20,10,ft_transformer
1,90,75,50,30,20,10,lr
2,75,90,50,30,20,10,rfc
3,75,90,50,30,20,10,xgbc
4,75,90,50,30,20,10,lgbm
5,90,75,50,30,20,10,svm


In [34]:
def getting_mean(df=shopper_df):
    percentiles_list = [10, 20, 30, 50, 75, 90]
    models_list = ['ft_transformer', 'lr', 'rfc', 'xgbc', 'lgbm', 'svm']
    all_df = list()
    shopper_df_float_func = shopper_df.select_dtypes(include=['float64']).columns
    # shopper_df_float_func = ['mi_f1_score', 'mrmr_f1_score', 'mi_mrmr_f1_score', 'pca_f1_score', 'mi_auc', 'mrmr_auc', 'mi_mrmr_auc', 'pca_auc']
    
    print(shopper_df_float_func)
    for j in percentiles_list:
        means = list()
        for i in models_list:
            model = df[(df.percentiles == j) & (df.models == i)]
            mean_val = list(model[shopper_df_float_func].mean())
            means.append(mean_val)
        means_df = pd.DataFrame(means, columns=shopper_df_float_func).reset_index()
        means_df['models_name'] = pd.Series(['ft_transformer', 'lr', 'rfc', 'xgbc', 'lgbm', 'svm'])
        means_df['percentiles_name'] = pd.Series([j] * 6)
    
        all_df.append(means_df)
    all_df = pd.concat(all_df)
    all_df.drop(['index', 'models_name', 'percentiles_name'], axis=1, inplace=True)
    return all_df

In [35]:
feature_reduction_df = getting_mean()
feature_reduction_df_f1 = feature_reduction_df[['mi_f1_score', 'mrmr_f1_score', 'mi_mrmr_f1_score', 'pca_f1_score']].reset_index(drop=True).T
feature_reduction_df_auc = feature_reduction_df[['mi_auc', 'mrmr_auc', 'mi_mrmr_auc', 'pca_auc']].reset_index(drop=True).T
feature_reduction_df_accuracy = feature_reduction_df[['mi_accuracy', 'mrmr_accuracy', 'mi_mrmr_accuracy', 'pca_accuracy']].reset_index(drop=True).T

Index(['mi_accuracy', 'mi_f1_score', 'mi_auc', 'mrmr_accuracy',
       'mrmr_f1_score', 'mrmr_auc', 'mi_mrmr_accuracy', 'mi_mrmr_f1_score',
       'mi_mrmr_auc', 'pca_accuracy', 'pca_f1_score', 'pca_auc'],
      dtype='object')


In [36]:
print(mcdm.rank(feature_reduction_df_f1.to_numpy(), s_method='TOPSIS', alt_names=['mi_f1_score', 'mrmr_f1_score', 'mi_mrmr_f1_score', 'pca_f1_score']))
print(mcdm.rank(feature_reduction_df_auc.to_numpy(), s_method='TOPSIS', alt_names=['mi_auc', 'mrmr_auc', 'mi_mrmr_auc', 'pca_auc']))
print(mcdm.rank(feature_reduction_df_accuracy.to_numpy(), s_method='TOPSIS', alt_names=['mi_accuracy', 'mrmr_accuracy', 'mi_mrmr_accuracy', 'pca_accuracy']))

[('mrmr_f1_score', 0.569688794245731), ('mi_f1_score', 0.5424005651434257), ('pca_f1_score', 0.49327948669257043), ('mi_mrmr_f1_score', 0.3066225816292378)]
[('mrmr_auc', 0.5695275147055034), ('mi_auc', 0.518317420759402), ('pca_auc', 0.4934681668554115), ('mi_mrmr_auc', 0.25937819676406193)]
[('mrmr_accuracy', 0.5594743539120962), ('mi_accuracy', 0.5357194111366047), ('pca_accuracy', 0.48894826404502634), ('mi_mrmr_accuracy', 0.299474097367262)]
